In [3]:
# Install required packages
%pip install openai==1.12.0 azure-kusto-data langchain tenacity langchain-openai

# Import libraries
from pyspark.sql import SparkSession
from notebookutils import mssparkutils
from openai import AzureOpenAI
import pandas as pd
import json
from tenacity import retry, wait_random_exponential, stop_after_attempt

# Restart the kernel after installation if you get any import errors

StatementMeta(, 7206fc12-bc87-410f-b591-2e3863c232ba, 10, Finished, Available, Finished)

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 63.5/63.5 kB 316.7 kB/s eta 0:00:00a 0:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 81.3/81.3 kB 1.3 MB/s eta 0:00:00a 0:00:01
INFO: pip is looking at multiple versions of langchain-openai to determine which version is compatible with other requirements. This could take a while.
INFO: pip is still looking at multiple versions of langchain-openai to determine which version is compatible with other requirements. This could take a while.
INFO: This is taking longer than usual. You might need to provide the dependency resolver with stricter constraints to reduce runtime. See https://pip.pypa.io/warnings/backtracking for guidance. If you want to abort this run, press Ctrl + C.
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 41.8/41.8 kB 2.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 226.7/226.7 kB 1.2 MB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 52.2/52.2 kB 5.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━

In [5]:
# Configuration
OPENAI_GPT4_DEPLOYMENT_NAME = "gpt-4"
OPENAI_DEPLOYMENT_ENDPOINT = mssparkutils.credentials.getSecret("openai-scope", "OPENAI_ENDPOINT")
OPENAI_API_KEY = mssparkutils.credentials.getSecret("openai-scope", "OPENAI_KEY")
OPENAI_ADA_EMBEDDING_DEPLOYMENT_NAME = "text-embedding-ada-002"

KUSTO_URI = "https://trd-zdxwqrcu1znbqygpxg.z2.kusto.fabric.microsoft.com"
KUSTO_DATABASE = "BioEventHouse"
KUSTO_TABLE = "biospecimen_embeddings"
accessToken = mssparkutils.credentials.getToken(KUSTO_URI)

client = AzureOpenAI(
    azure_endpoint=OPENAI_DEPLOYMENT_ENDPOINT,
    api_key=OPENAI_API_KEY,
    api_version="2023-09-01-preview"
)

@retry(wait=wait_random_exponential(min=1, max=20), stop=stop_after_attempt(6))
def generate_embeddings(text):
    txt = text.replace("\n", " ")
    return client.embeddings.create(input=[txt], model=OPENAI_ADA_EMBEDDING_DEPLOYMENT_NAME).data[0].embedding

# Data Preparation
def prepare_data():
    # Read from Lakehouse
    df = spark.read.format("csv").option("header", "true").load("/lakehouse/default/Files/biospecimen_data.csv")
    
    # Convert to pandas for easier processing
    pdf = df.toPandas()
    
    # Create document text
    def create_document_text(row):
        text_parts = []
        if pd.notna(row['Sample Type']):
            text_parts.append(f"Sample Type: {row['Sample Type']}")
        if pd.notna(row['Primary Site']):
            text_parts.append(f"Primary Site: {row['Primary Site']}")
        # Add other fields as shown in previous example
        return ". ".join(text_parts)
    
    pdf['document_text'] = pdf.apply(create_document_text, axis=1)
    
    # Generate embeddings
    pdf['embedding'] = pdf['document_text'].apply(lambda x: generate_embeddings(x))
    
    # Prepare for Kusto
    embeddings_df = pd.DataFrame({
        'document_id': pdf['Aliquot ID'],
        'content': pdf['document_text'],
        'metadata': pdf.apply(lambda x: {
            'sample_type': x['Sample Type'],
            'primary_site': x['Primary Site'],
            'case_id': x['Case Submitter ID']
        }, axis=1),
        'embedding': pdf['embedding']
    })
    
    return spark.createDataFrame(embeddings_df)

# Store in Eventhouse
def store_embeddings(embeddings_df):
    embeddings_df.write.\
        format("com.microsoft.kusto.spark.synapse.datasource").\
        option("kustoCluster", KUSTO_URI).\
        option("kustoDatabase", KUSTO_DATABASE).\
        option("kustoTable", KUSTO_TABLE).\
        option("accessToken", accessToken).\
        mode("Append").save()

# Query System
def query_biospecimen_data(question, nr_of_answers=3):
    searchedEmbedding = generate_embeddings(question)
    
    kusto_query = f"""
    {KUSTO_TABLE} 
    | extend similarity = series_cosine_similarity(dynamic({str(searchedEmbedding)}), embedding) 
    | top {nr_of_answers} by similarity desc
    | project content, metadata, similarity
    """
    
    kustoDf = spark.read\
        .format("com.microsoft.kusto.spark.synapse.datasource")\
        .option("kustoCluster", KUSTO_URI)\
        .option("kustoDatabase", KUSTO_DATABASE)\
        .option("accessToken", accessToken)\
        .option("kustoQuery", kusto_query).load()
    
    results = [row.asDict() for row in kustoDf.collect()]
    
    # Prepare context for LLM
    context = "\n\n".join([f"Record {i+1} (Similarity: {r['similarity']:.2f}):\n{r['content']}\nMetadata: {r['metadata']}" 
                          for i, r in enumerate(results)])
    
    prompt = f"""
    You are a biomedical research assistant analyzing breast cancer biospecimen data.
    Answer the user's question based on the following records. Be precise and cite specific records when relevant.
    
    Question: {question}
    
    Relevant Biospecimen Records:
    {context}
    """
    
    response = client.chat.completions.create(
        model=OPENAI_GPT4_DEPLOYMENT_NAME,
        messages=[
            {"role": "system", "content": "You are a knowledgeable biomedical research assistant."},
            {"role": "user", "content": prompt}
        ],
        temperature=0
    )
    
    return {
        "answer": response.choices[0].message.content,
        "sources": results
    }

# Initialize the system (run once)
embeddings_df = prepare_data()
store_embeddings(embeddings_df)
print("RAG system initialized successfully!")
# Type here in the cell editor to add code!


StatementMeta(, 7206fc12-bc87-410f-b591-2e3863c232ba, 13, Finished, Available, Finished)

Py4JJavaError: An error occurred while calling z:mssparkutils.credentials.getSecret.
: com.microsoft.azure.trident.tokenlibrary.util.AkvHttpClientException: Invalid vault uri. Uri should match azure key vault URI like https://<keyVaultName>.vault.azure.net/
	at com.microsoft.azure.trident.tokenlibrary.util.AkvBasedSecretProviderClientImpl.invokeGetTarget(AkvBasedSecretProviderClient.scala:122)
	at com.microsoft.azure.trident.tokenlibrary.util.AkvBasedSecretProviderClientImpl.getAkvSecretWithAccessToken(AkvBasedSecretProviderClient.scala:153)
	at com.microsoft.azure.trident.tokenlibrary.TokenLibrary.getSecretWithToken(TokenLibrary.scala:806)
	at com.microsoft.azure.trident.tokenlibrary.TokenLibrary$.getSecretWithToken(TokenLibrary.scala:1359)
	at mssparkutils.credentials$.getSecret(credentials.scala:166)
	at mssparkutils.credentials.getSecret(credentials.scala)
	at java.base/jdk.internal.reflect.NativeMethodAccessorImpl.invoke0(Native Method)
	at java.base/jdk.internal.reflect.NativeMet